# Lecture 7: Dimensionality Reduction and Clustering

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups, load_digits
from sklearn.feature_extraction.text import TfidfVectorizer

# these are new imports for dimensionality reduction
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
# these are new imports for clustering
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage, dendrogram

## Dimensionality Reduction

### PCA

The R package [`cluster.datasets`](http://cran.r-project.org/web/packages/cluster.datasets/cluster.datasets.pdf) has some good datasets for experimenting with unsupervised learning techniques like dimensionality reduction and clustering.  Here, we'll use the `cake.ingredients.1961` dataset of cake recipes, which I've exported to a CSV.

In [ ]:
# Load the cakes data, and take a look
cakes = pd.read_csv("./cakes.csv")
cakes.head()

Each row is a cake recipe, and each column is an ingredient

In [ ]:
# Let's remove leading and trailing spaces from the 'Cake' column to clean up names of cakes
cakes["Cake"] = cakes.Cake.str.strip() # Remove leading
cakes.head()

Let's store a dictionary of the ingredient abbreviations so we can look them up:

In [ ]:
ingredients_dict = {
    "AE": "Almond essence",
    "BM": "Buttermilk",
    "BP": "Baking powder",
    "BR": "Butter",
    "BS": "Bananas",
    "CA": "Cocoa",
    "CC": "Cottage Cheese",
    "CE": "Chocolate",
    "CI": "Crushed Ice",
    "CS": "Crumbs",
    "CT": "Cream of tartar",
    "DC": "Dried currants",
    "EG": "Eggs",
    "EY": "Egg white",
    "EW": "Egg yolk",
    "FR": "Sifted flour",
    "GN": "Gelatin",
    "HC": "Heavy cream",
    "LJ": "Lemon juice",
    "LR": "Lemon",
    "MK": "Milk",
    "NG": "Nutmeg",
    "NS": "Nuts",
    "RM": "Rum",
    "SA": "Soda",
    "SC": "Sour cream",
    "SG": "Shortening",
    "SR": "Granulated sugar",
    "SS": "Strawberries",
    "ST": "Salt",
    "VE": "Vanilla extract",
    "WR": "Water",
    "YT": "Yeast",
    "ZH": "Zwiebach"
}

Get rid of the column of cake names so that we have a numeric only dataframe:

In [ ]:
X = cakes.iloc[:, 1:]
X

In [ ]:
X.shape

First, we'll run a simple PCA using the [scikit-learn class](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html).  If we don't specify `n_components` or set it to `None`, it will use the maximum number of principal components.  Note that the principal components are unique up to a sign, so on a small data set it makes sense to go ahead and get all the components. If you have a large data set, you may want to just get the first couple of components, then decide if you need more computation to get the remaining components.

In [ ]:
pca = PCA(n_components=None)
pca.fit(X)

Let's take a look at the explained variance of each of the principal components.

In [ ]:
pca.explained_variance_ratio_

And plot it:

In [ ]:
# Line plot of variance explained
plt.plot(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_) 

# Add points to the plot
plt.scatter(range(len(pca.explained_variance_ratio_)), pca.explained_variance_ratio_) 

# Axes labels
plt.xlabel("Principal Components Number")
plt.ylabel("Percentage of Variance Explained")
plt.show()

In [ ]:
# Now, let's lot the cumulative variance explained
cumsum = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(len(pca.explained_variance_ratio_)), cumsum)
plt.scatter(range(len(pca.explained_variance_ratio_)), cumsum)
plt.xlabel("Principal Components Number")
plt.ylabel("Cumulative Percentage of Variance Explained")
plt.show()

If we're looking for an "elbow", it looks like roughly 6 or 7 principal components would be enough.  To actually get each row transformed into the principal component space, we can call `transform` on an already fit `PCA` object, or we can do both at once with `fit_transform`:

In [ ]:
# We'll redo PCA, but only with 2 components
pca = PCA(n_components=2)
X_trans = pca.fit_transform(X)
X_trans

The first column represents the **scores** of each observation for the first component, and the second column represents the scores for the second component.

Next, let's define a function for plotting. This function will let us plot the scores for each observation, and optionally the loadings for each of the components.  If we include the loadings, we end up with a *biplot*.

In [ ]:
def plot_PCA(pca, X, print_row_labels, row_labels, col_labels, biplot=False, y_scale=(None, None), font_size=None):
    
    # transform our data to PCA space
    X_trans = pca.fit_transform(X)

    # handle the scaling of the plot
    xmin, xmax = min(X_trans[:, 0]), max(X_trans[:, 0])
    if y_scale == (None, None): # use the data to determine the scale on the vertical axis.
        ymin, ymax = min(X_trans[:, 1]), max(X_trans[:, 1])
        xpad, ypad = 5, 5
    else: # Otherwise, use the vertical scale passed into the function
        ymin, ymax = y_scale
        xpad, ypad = 5, 1
        
    plt.xlim(xmin - xpad, xmax + xpad) # Set the horizontal limits
    plt.ylim(ymin - ypad, ymax + ypad) # Set the vertical limits

    # plot words instead of points
    # We use 'zip' to create a collecton of tuples, one per observation, that
    # collects information on that tuple -score for first prin. component, score for second component, and text label
    if print_row_labels:
        for x, y, label in zip(X_trans[:, 0], X_trans[:, 1], row_labels):
            if font_size is None:
                plt.text(x, y, label)
            else:
                plt.text(x, y, label, size=font_size)
    else:
        for x, y in zip(X_trans[:, 0], X_trans[:, 1]):
            plt.scatter(x, y)
    plt.xlabel("PC 1")
    plt.ylabel("PC 2")

    # if we want a biplot, get the loading and plot
    # axes with labels
    if biplot:
        eigenvectors = pca.components_.transpose()
        for i, col in enumerate(col_labels):
            x, y = 10*eigenvectors[i][0], 10*eigenvectors[i][1]
            plt.arrow(0, 0, x, y, color='r', width=0.002, head_width=0.05)
            plt.text(x* 1.4, y * 1.4, col, color='r', ha='center', va='center')
    
    plt.show()

In [ ]:
#Let's plot jsut the scores for each observation. 
plot_PCA(pca, X, True, cakes.Cake, X.columns, biplot=False)

Not encouraging.  When we see something like this, it's typically a scaling issue.  Let's plot again, but include the loadings.

In [ ]:
plot_PCA(pca, X, True, cakes.Cake, X.columns, biplot=True)

We see that we're influenced by two large outliers - 'One Bowl Chocolate' and 'Angel' cake.    The first principal component is dominated by "cocoa" and "shortening" because the "One Bowl Chocolate" cake has a huge amount of these.  The second principal component is dominated by "egg whites" because of the "Angel" foodcake recipe.

To resolve this issue, let's first try mean-centering the columns.

In [ ]:
pca = PCA(n_components=2)
X_scaled = scale(X, with_mean=True, with_std=False) # Mean-centering
plot_PCA(pca, X_scaled, True, cakes.Cake, X.columns, biplot=True)

And now both center and scale to unit variance:

In [ ]:
pca = PCA(n_components=2)
X_scaled = scale(X, with_mean=True, with_std=True) # Center and scale
plot_PCA(pca, X_scaled, True, cakes.Cake, X.columns, biplot=True)

Let's look at a zoomed in version:

In [ ]:
X_scaled

In [ ]:
pca = PCA(n_components=2)
X_scaled = scale(X, with_mean=True, with_std=True)
plot_PCA(pca, X_scaled, True, cakes.Cake, X.columns, biplot=True, y_scale=(-1, 1))

To me, it looks like cheesecakes are off to the right on the first principal component, and the second principal component is quantifying whether the cake has fruit or not...but, this is subjective.

### t-SNE

Let's switch for a moment to dataset on handwritten digits.  Each observation is an 8 pixel by 8 pixel image, so each image is characterized by 64 pieces of information (color for each pixel).  We also get data on what each image represents (a digit between 0 and 9), though we won't use this for the purposes of unsupervised learning. We will use the target data to understand how well the PCA is doing for teaching purposes.

In [ ]:
digits = load_digits()
digits.data.shape

In [ ]:
# this will show us the pixel values
image_num = 1000
digits.images[image_num]

In [ ]:
# We can also see the labels for each images, i.e., the digit in each image.
digits.target

We can write a function to plot each image.

In [ ]:
def plot_handwritten_digit(the_image, label):
    plt.axis('off')
    plt.imshow(the_image, cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('Image: %i' % label)

Now we plot one of the images.

In [ ]:
# Generates the image, with the digit label listed in the title
plot_handwritten_digit(digits.images[image_num], digits.target[image_num])

In [ ]:
# Now, we'll scale the digits data and store the labels.
digits_data = scale(digits.data)

labels = digits.target

In [ ]:
digits_data.shape

Now, we'll run PCA on the digits data with two components.

In [ ]:
pca = PCA(n_components=2)
digits_trans = pca.fit_transform(digits_data)

Let's make a plot of the first two principal components, colored and labeled by the true digit:

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

for x, y, label in zip(digits_trans[:, 0], digits_trans[:, 1], labels):
    plt.text(x, y, label, size=8, color=plt.cm.Set1(label/10.))

plt.xlabel("PC 1")
plt.ylabel("PC 2")

plt.show()

Not the clearest view of the data.

If our goal is visualizing a high dimensional dataset, the [t-SNE](http://lvdmaaten.github.io/tsne/) algorithm usually does a superior job of finding structure in the high-dimensional data that can be visualized in two dimensions. t-SNE works by trying to find a lower dimensional embedding of the data such that observations near each other in lower-dimensional space were close to each other in higher dimensional space.  It is an example of a non-linear dimensionality reduction technique.  Another example of such a technique is a neural network autoencoder (which I'll touch on next lecture).

t-SNE is often used in one of two ways.  First, you can apply it before clustering to get a good 2-d or 3-d representation of the data, *then* apply clustering to the 2-d or 3-d projection of the data.  Second, you can cluster the data first, then apply t-SNE just to get a better visualization of the data.

There's a [scikit-learn class](http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE) for running t-SNE.  

In [ ]:
tsne = TSNE(n_components=2, verbose=True)
digits_trans = tsne.fit_transform(digits_data)

In [ ]:
# Like PCA with two components, TSNE also describes each observation using two coordinates.
# These are analogous to, but derived differently from, the first two PCA components
digits_trans[0:5,]

In [ ]:
# We can again plot the data, this time using the two-dimensional projection
# provided by TSNE

xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

#for x, y, label in zip(digits_trans[labels==6, 0], digits_trans[labels==6, 1], labels[labels==6]):
for x, y, label in zip(digits_trans[0:1000, 0], digits_trans[0:1000, 1], labels[0:1000]):
    plt.text(x, y, label, size=8, color=plt.cm.Set1(label/10.))

plt.xlabel("Component 1")
plt.ylabel("Component 2")

plt.show()

This clearly does a better job at finding the "structure" in the high-dimensional dataset.  Notice that 3, 5, and 9 end up near each other.  But there are some 1's that are closer to the 2's, and some 9's that are closer to the 7's and the 1's.

## Clustering

### Hierarchical

Now back to cakes.  We'll use some functions from scipy to run hierarchical clustering.  `linkage` calculates the distances and linkages, and `dendrogram` displays the actual tree dendrogram:

In [ ]:
clusters_single = linkage(scale(X), method='single', metric="euclidean") # single, complete, average, and ward methods

In [ ]:
dendr = dendrogram(clusters_single, orientation="top", labels=list(cakes.Cake))

As ISLR says, single linkage tends to produce really unbalanced trees.  We can put the dendrogram on its side to make it easier to visualize:

In [ ]:
dendr = dendrogram(clusters_single, orientation="right", labels=list(cakes.Cake))

In [ ]:
clusters_complete = linkage(scale(X), method='complete', metric="euclidean") # single, complete, average, and ward methods

In [ ]:
dendr = dendrogram(clusters_complete, orientation="top", labels=list(cakes.Cake))

In [ ]:
dendr = dendrogram(clusters_complete, orientation="right", labels=list(cakes.Cake))

Another linkage method is **Ward's Method**, which is slightly diffferent than the other methods. It decides which clusters to merge by, at each step, considering every possible merge of two clusters and using the one that least increases the total within-cluster variation.  This resembles the greedy approach taken by decision treess in it uses a greedy approach to minimzing an objective function that measures error.

In [ ]:
clusters_ward = linkage(scale(X), method='ward', metric="euclidean") # single, complete, average, and ward methods

In [ ]:
dendr = dendrogram(clusters_ward, orientation="top", labels=list(cakes.Cake))

In [ ]:
dendr = dendrogram(clusters_ward, orientation="right", labels=list(cakes.Cake))

The clustering is doing something sensible: the cheesecakes group together and are on their own, the chocolate cakes are together (sour cream fudge, red devil's, sweet chocolate, and one bowl chocolate), etc.

### k-Means

As a general resource, the [scikit-learn clustering page](http://scikit-learn.org/stable/modules/clustering.html) is great.  It has all the different kinds of clustering algorithms with their pros and cons.  Here, we'll focus on k-means for clustering the digits data.

In [ ]:
# init can be k-means++ or random; k-means++ is just a smarter version of random that forces the
# centers to be further apart
kmeans = KMeans(n_clusters=10, init='k-means++', n_init=10, max_iter=300, verbose=True, n_jobs=1)

In [ ]:
kmeans.fit(digits_data)

We can see the assigned cluster or label of each data point:

In [ ]:
kmeans.labels_

And the cluster centers themselves.  Each centroid is a vector with 64 values -one per pixel in an 8x8 pixel image.

In [ ]:
kmeans.cluster_centers_

The "inertia" tells us the within cluster sum-of-squares, or the "sum of distances of samples to their closest cluster center."

In [ ]:
kmeans.inertia_

Here, we make a plot where we color by the k-means label instead of the true label.  In order to get a two dimensional plot, we plot each observation in terms of the t-SNE projection from earlier in this notebook.  However, we've still clustered in the original 64-dimensional space -therefore, this is an example of using t-SNE to help visualize data that's already been clustered.

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

for x, y, true_label, kmeans_label in zip(digits_trans[:, 0], digits_trans[:, 1], labels, kmeans.labels_):
    plt.text(x, y, true_label, size=8, color=plt.cm.Set1(kmeans_label/10.))

plt.xlabel("Component 1")
plt.ylabel("Component 2")

plt.show()

We can see that things are decent, but definitely more confused than with the true labels.  Ideally, all there should only be one color per digit label.

We can call the `predict` method, which will tell us which cluster center some new data is closest too:

In [ ]:
kmeans.predict(digits_data)

The `transform` method will transform data into the cluster distance space.  That is, how far the point is from each cluster center.  Hence, the resulting object ```transformed``` will have one row per observation, and one column per cluster.

In [ ]:
transformed = kmeans.transform(digits_data)
transformed[0, :]

For very large datasets, there's a much faster implementation of k-means called [mini-batch k-means](http://www.eecs.tufts.edu/~dsculley/papers/fastkmeans.pdf), and a [scikit-learn class](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.MiniBatchKMeans.html) for running it:

In [ ]:
mb_kmeans = MiniBatchKMeans(n_clusters=10, batch_size=100, init='k-means++', n_init=10, max_iter=300, verbose=True)

In [ ]:
mb_kmeans.fit(digits_data)

In [ ]:
mb_kmeans.labels_

In [ ]:
xmin, xmax = min(digits_trans[:, 0]), max(digits_trans[:, 0])
ymin, ymax = min(digits_trans[:, 1]), max(digits_trans[:, 1])
xpad, ypad = 5, 5
plt.xlim(xmin - xpad, xmax + xpad)
plt.ylim(ymin - ypad, ymax + ypad)

for x, y, true_label, kmeans_label in zip(digits_trans[:, 0], digits_trans[:, 1], labels, mb_kmeans.labels_):
    plt.text(x, y, true_label, size=8, color=plt.cm.Set1(kmeans_label/10.))

plt.xlabel("Component 1")
plt.ylabel("Component 2")

plt.show()

Let's see if we can re-cover the "correct" number of clusters using the silhouette statistic:

In [ ]:
n_clusters = range(3, 70, 2)
silhouette_stats = []
for this_n_clusters in n_clusters:
    print "Fitting %s clusters..." % this_n_clusters
    kmeans = KMeans(n_clusters=this_n_clusters, init='k-means++', n_init=10, max_iter=300, verbose=False, n_jobs=1)
    kmeans.fit(digits_data)
    labels = kmeans.labels_
    silhouette_stats.append(silhouette_score(digits_data, labels, metric='euclidean'))

In [ ]:
plt.plot(n_clusters, silhouette_stats)
plt.show()